In [8]:
import Py6S
import numpy as np
from pyeosim.spectral import TreeView_3
import xarray

In [2]:
tv3 = TreeView_3()

In [4]:
a.geometry.solar_z

33.286874248758366

In [5]:
a.run()

In [30]:
def generate_mk_test_sixs():
    a = Py6S.SixS()

    a.ground_reflectance = Py6S.GroundReflectance.HomogeneousLambertian(.2)
    a.altitudes.set_sensor_satellite_level()
    geom = Py6S.Geometry.User()
    geom.from_time_and_location(lat=52.04,
                                lon=0.76,
                                datetimestring='2020-06-22 10:30:00',
                                view_z=0,
                                view_a=0)
    a.geometry = geom
    return a

def get_correction_coefs(fitted_6s):
    s = fitted_6s
    # solar irradiance
    # direct solar irradiance
    Edir = s.outputs.direct_solar_irradiance
    # diffuse solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance
    # sum for total
    E = Edir + Edif                                      
    # transmissivity
    # absorption transmissivity
    absorb  = s.outputs.trans['global_gas'].upward
    # scattering transmissivity
    scatter = s.outputs.trans['total_scattering'].upward
    # transmissivity (from surface to sensor)
    tau = absorb * scatter
    # path radiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance
    
    # correction coefficients for this configuration
    # i.e. surface_reflectance = (L - a) / b,
    #      where, L is at-sensor radiance
    a = Lp
    b = (tau * E) / np.pi
    return a, b

def generate_correction_LUT(six_s_instance, spectral_responses):
    """
    generate the LUT for a sensor given a parameterised SixS object
    
    Parameters
    ----------
    SixS : Py6S.SixS
        atmospheric conditions
    spectral_responses : pyeosim.spectral
        spectral response object
    """
    a = []
    b = []
    srfs = spectral_responses.to_6sv().values()
    for srf in srfs:
        six_s_instance.wavelength = Py6S.Wavelength(*srf)
        six_s_instance.run()
        _a, _b = get_correction_coefs(six_s_instance)
        a.append(_a)
        b.append(_b)
        
    # convert to xarray
    a = xarray.DataArray(a, coords=[('band', np.arange(len(srfs)))])
    b = xarray.DataArray(b, coords=[('band', np.arange(len(srfs)))])
    return a,b


In [31]:
v = generate_correction_LUT(a, TreeView_3())

In [35]:
xarray.Dataset({'a':v[0], 'b':v[1]})

<xarray.Dataset>
Dimensions:  (band: 8)
Coordinates:
  * band     (band) int64 0 1 2 3 4 5 6 7
Data variables:
    a        (band) float64 61.42 38.63 27.61 22.5 18.66 13.77 12.51 9.055
    b        (band) float64 319.6 364.4 344.2 334.8 310.7 266.4 269.2 222.5

<xarray.DataArray (band: 8)>
array([61.424, 38.635, 27.61 , 22.499, 18.662, 13.774, 12.509,  9.055])
Coordinates:
  * band     (band) int64 0 1 2 3 4 5 6 7

In [12]:
list({'a':1}.values())

[1]

In [7]:
get_correction_coefs(a)

(61.424, 319.6447810335479)